## Step 1: Fetch Google Street View Metadata

This module retrieves image metadata (e.g., `pano_id`, `date`, `status`)  
using the [Google Street View Metadata API](https://developers.google.com/maps/documentation/streetview/metadata).

You will need:
- A valid **Google Street View API key**
- An input CSV file containing at least these columns: `latitude`, `longitude`


The output will be a CSV file storing metadata for all queryable points, including:
- Latitude / Longitude
- Panorama ID (`pano_id`)
- Capture date
- Status (`OK`, `ZERO_RESULTS`, etc.)


In [1]:
import requests
import json
import os
import csv

In [ ]:
class MetaDataFetcher(object):
    def __init__(self, api_key, latitude, longitude, heading=0, verbose=True):
        """
        This class handles fetching metadata from Google Street View API
        api_key: Google Cloud Platform API key
        latitude, longitude: target coordinates
        heading: camera direction
        verbose: whether to print the processing status
        """
        self.api_key = api_key
        self.latitude = latitude
        self.longitude = longitude
        self.heading = heading
        self.verbose = verbose

        # Define API parameters
        self._meta_params = {
            'key': self.api_key,
            'location': f"{self.latitude},{self.longitude}",
            'size': "640x640",  # Image size (irrelevant for metadata API)
            'heading': self.heading,  # Camera direction
            'fov': 90,  # Field of View
            'pitch': 0,  # Vertical angle
            'radius': 5,  # Search radius
            'source': 'outdoor',  # Limits searches to outdoor collections.
            'return_error_code': 'true'  # Whether to return error codes
        }

    def get_meta(self):
        """
        Method to query metadata
        """
        # Send request to metadata API
        response = requests.get(
            'https://maps.googleapis.com/maps/api/streetview/metadata?',
            params=self._meta_params
        )
        if response.ok:
            self.meta_info = response.json()
            self.meta_status = self.meta_info['status']
            if self.verbose:
                print(">>> Meta Data Retrieved:")
                print(self.meta_info)
            return self.meta_info
        else:
            print(">>> Failed to retrieve metadata!")
            self.meta_status = 'FAILED'
        response.close()
        return None

    def save_meta_to_csv(self, meta_info, csv_filename='meta_data.csv'):
        """
        Save the meta information to a CSV file
        """
        # Define the CSV fieldnames (columns)
        fieldnames = ['copyright', 'date', 'latitude', 'longitude', 'pano_id', 'status']

        # Prepare data to be written to CSV
        row = {
            'copyright': meta_info.get('copyright', ''),
            'date': meta_info.get('date', ''),
            'latitude': self.latitude,
            'longitude': self.longitude,
            'pano_id': meta_info.get('pano_id', ''),
            'status': meta_info.get('status', 'UNKNOWN')
        }

        # Open the CSV file in append mode and write the row data
        with open(csv_filename, mode='a', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=fieldnames)

            # If the file is empty, write the header first
            if file.tell() == 0:
                writer.writeheader()

            # Write the data row
            writer.writerow(row)

        print(f"Metadata for ({self.latitude}, {self.longitude}) has been saved to {csv_filename}")

In [ ]:
# Input your Google Street View API key
# Note: The key will not be stored and must be kept private
API_KEY = input("Enter your Google Street View API key: ").strip()

# Input file: a CSV containing columns ['latitude', 'longitude']
csv_input_file = input("Enter the path to your input CSV file (with latitude, longitude columns): ").strip()

# Output file: path to save the retrieved metadata
output_csv_file = input("Enter the path to the output metadata CSV file: ").strip()

In [ ]:
# Read CSV file and fetch metadata for each row
with open(csv_input_file, mode='r', encoding='utf-8') as file:
    reader = csv.DictReader(file)

    for row in reader:
        latitude = float(row['latitude'])  # Get Latitude
        longitude = float(row['longitude'])  # Get Longitude

        # Create an instance of MetaDataFetcher and fetch metadata
        meta_fetcher = MetaDataFetcher(api_key=API_KEY, latitude=latitude, longitude=longitude)
        meta_data = meta_fetcher.get_meta()

        # If metadata is successfully retrieved, save it to the CSV file
        if meta_data and meta_data.get('status') == 'OK':
            meta_fetcher.save_meta_to_csv(meta_data, csv_filename=output_csv_file)
        else:
            print(f"No valid metadata found for ({latitude}, {longitude}).")
print("end")

## Step 2: Download Front & Back Street View Images

This step downloads front- and back-facing Street View images using `latitude`, `longitude`, and directional angles (`front`, `back`).

### Input CSV

Use a filtered version of the Step 1 output (rows with `status == OK`), with added `front` and `back` heading columns.

**Required columns:**
- `latitude`
- `longitude`
- `front`
- `back`

### Output

Each row will produce two images:
- `Front_<lon>_<lat>.jpg`
- `Back_<lon>_<lat>.jpg`

In [ ]:
# Input your API key (again, if not already saved)
API_KEY_IMG = input("Enter your Google Street View API key (for image download): ").strip()

# Path to the input CSV (must include latitude, longitude, front, back columns)
csv_img_input = input("Enter the path to your metadata CSV file: ").strip()

# Directory to save the downloaded images
output_dir = input("Enter the output directory where images will be saved: ").strip()

os.makedirs(output_dir, exist_ok=True)

In [5]:
# Helper to read CSV into point dicts
def read_csv(file_path):
    points = []
    with open(file_path, mode="r", encoding="utf-8") as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            points.append({
                "longitude": row.get("longitude"),
                "latitude": row.get("latitude"),
                "front_heading": row.get("front"),
                "back_heading": row.get("back")
            })
    return points

# Build URL parameters
def define_params(api_key, latitude, longitude, heading):
    return {
        'key': api_key,
        'location': f"{latitude},{longitude}",
        'size': "640x640",
        'heading': heading,
        'fov': 90,
        'pitch': 0,
        'radius': 10,
        'source': 'outdoor',
        'return_error_code': 'true'
    }

# Fetch response from GSV
def fetch_data(base_url, params):
    return requests.get(base_url, params=params)

# Save image if response is OK
def save_image(response, output_path):
    with open(output_path, "wb") as f:
        f.write(response.content)
    print(f"Saved image: {output_path}")

In [ ]:
# Base URL for image API
pic_base = 'https://maps.googleapis.com/maps/api/streetview?'

# Read points from CSV
points = read_csv(csv_img_input)

# Process each point
for i, point in enumerate(points):
    lat = point["latitude"]
    lon = point["longitude"]
    front_heading = point["front_heading"]
    back_heading = point["back_heading"]

    print(f"Processing Point {i+1}: Latitude={lat}, Longitude={lon}")

    # Front image
    front_path = os.path.join(output_dir, f"Front_{lon}_{lat}.jpg")
    if not os.path.exists(front_path):
        params_front = define_params(API_KEY_IMG, lat, lon, front_heading)
        res_front = fetch_data(pic_base, params_front)
        if res_front.status_code == 200:
            save_image(res_front, front_path)
        else:
            print(f"Failed to get front image for {lon}, {lat}")
    else:
        print(f"Front image already exists for {lon}, {lat}")

    # Back image
    back_path = os.path.join(output_dir, f"Back_{lon}_{lat}.jpg")
    if not os.path.exists(back_path):
        params_back = define_params(API_KEY_IMG, lat, lon, back_heading)
        res_back = fetch_data(pic_base, params_back)
        if res_back.status_code == 200:
            save_image(res_back, back_path)
        else:
            print(f"Failed to get back image for {lon}, {lat}")
    else:
        print(f"Back image already exists for {lon}, {lat}")

print("All images processed.")